In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import acquire
#import prepare
from scipy import stats

from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

#from sklearn.model_selection import train_test_split
#from sklearn.impute import SimpleImputer

# 2. Given the following confusion matrix, evaluate (by hand) the model's performance.

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |


In [1]:
#ACCURACY
(46 + 34) / (46 + 7 + 13 + 34)

0.8

In [7]:
#PRECISION
(46) / (46 + 13)

0.7796610169491526

In [6]:
#RECALL
(46) / (46 + 7)

0.8679245283018868

2a. In the context of this problem, what is a false positive?

In [ ]:
#FP: Predicting they like cats, actuality they like dogs.

2b. In the context of this problem, what is a false negative?

In [ ]:
#FN: Predicting they like dogs, actuality they like cats.

2c. How would you describe this model?

In [ ]:
#As a Pandas Crosstab confusion_matrix

# 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.
Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.: https://ds.codeup.com/data/c3.csv

Use the predictions dataset and pandas to help answer the following questions:

In [4]:
c3 = pd.read_csv('c3.csv')
c3.head(2)

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect


In [23]:
pd.crosstab(c3.actual, c3.model1)

model1,Defect,No Defect
actual,,
Defect,8,8
No Defect,2,182


In [24]:
pd.crosstab(c3.actual, c3.model2)

model2,Defect,No Defect
actual,,
Defect,9,7
No Defect,81,103


In [25]:
pd.crosstab(c3.actual, c3.model3)

model3,Defect,No Defect
actual,,
Defect,13,3
No Defect,86,98


#### 3a. An internal team wants to investigate the cause of the manufacturing defects. 
They tell you that they want to identify as many of the ducks that have a defect as possible. 
- *Which* evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [11]:
c3.model1.value_counts(), c3.model2.value_counts(), c3.model3.value_counts()

(No Defect    190
 Defect        10
 Name: model1, dtype: int64,
 No Defect    110
 Defect        90
 Name: model2, dtype: int64,
 No Defect    101
 Defect        99
 Name: model3, dtype: int64)

In [ ]:
# Metric: Precision b'c they want to predict as many defective ducks as possible
# Model: use model1 has the highest Precision %

In [37]:
c3['baseline_prediction'] = 'Defect'

In [38]:
c3.head()

,actual,model1,model2,model3,baseline_prediction
0,No Defect,No Defect,Defect,No Defect,Defect
1,No Defect,No Defect,Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect,Defect
3,No Defect,Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect,Defect


In [51]:
model_accuracy = (c3.model1 == c3.actual).mean()
baseline_accuracy = (c3.baseline_prediction == c3.actual).mean()

print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model accuracy: 95.00%
baseline accuracy: 8.00%


*Recall* is how well we do on actually positive cases. Here we'll define positive as **defect**.

First we'll subset the dataframe so that we are only looking at the rows where we have the positive case. 
- Then we'll evaluate how well our model's predictions do.

In [43]:
subset = c3[c3.actual == 'Defect']

In [44]:
subset.head()

,actual,model1,model2,model3,baseline_prediction
13,Defect,No Defect,Defect,Defect,Defect
30,Defect,Defect,No Defect,Defect,Defect
65,Defect,Defect,Defect,Defect,Defect
70,Defect,Defect,Defect,Defect,Defect
74,Defect,No Defect,No Defect,Defect,Defect


In [52]:
model_recall = (subset.model1 == subset.actual).mean()
baseline_recall = (subset.baseline_prediction == subset.actual).mean()

print(f'   model recall: {model_recall:.2%}')# .2% is simply rounding the decimal on the percentage 2 decimal places.
print(f'baseline recall: {baseline_recall:.2%}')

   model recall: 95.00%
baseline recall: 8.00%


Next we'll calculate precision. 

Precision is based on just the times that the model predicts the positive class. 

- Because the predictions for our model and the baseline differ, we'll need to create 2 seperate subsets here.

In [53]:
subset = c3[c3.model1 == 'Defect']
model_precision = (subset.model1 == subset.actual).mean()

subset = c3[c3.baseline_prediction == 'Defect']
baseline_precision = (subset.baseline_prediction == subset.actual).mean()

print(f'model precision: {model_precision:.2%}')
print(f'baseline precision: {baseline_precision:.2%}')

model precision: 80.00%
baseline precision: 8.00%


### Using sklearn Metrics

In [12]:
y_true = c3.actual
y_pred1 = c3.model1
y_pred2 = c3.model2
y_pred3 = c3.model3

In [17]:
accuracy_score(y_true, y_pred1), accuracy_score(y_true, y_pred2), accuracy_score(y_true, y_pred3)

(0.95, 0.56, 0.555)

In [41]:
precision_score(y_true, y_pred1, labels = ['No Defect', 'Defect'], average = None), precision_score(y_true, y_pred2, labels = ['No Defect', 'Defect'], average = None), precision_score(y_true, y_pred3, labels = ['No Defect', 'Defect'], average = None)

(array([0.95789474, 0.8       ]),
 array([0.93636364, 0.1       ]),
 array([0.97029703, 0.13131313]))

In [42]:
recall_score(y_true, y_pred1, labels = ['No Defect', 'Defect'], average = None), recall_score(y_true, y_pred2, labels = ['No Defect', 'Defect'], average = None), recall_score(y_true, y_pred3, labels = ['No Defect', 'Defect'], average = None)

(array([0.98913043, 0.5       ]),
 array([0.55978261, 0.5625    ]),
 array([0.5326087, 0.8125   ]))

In [49]:
print(classification_report(y_true, y_pred1, labels = ['No Defect', 'Defect']))

              precision    recall  f1-score   support

   No Defect       0.96      0.99      0.97       184
      Defect       0.80      0.50      0.62        16

    accuracy                           0.95       200
   macro avg       0.88      0.74      0.79       200
weighted avg       0.95      0.95      0.94       200



#### 3b. Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. 
The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. 

They need you to predict which ducks will have defects, but tell you they really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. 
- Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [50]:
# Metric: Recall b'c FN would be a prediction of no_defect & actuality defect = COSTLY.
# Model: use model3 as it has the most amt of defects

In [55]:
subset = c3[c3.actual == 'Defect']

In [56]:
subset.head()

,actual,model1,model2,model3,baseline_prediction
13,Defect,No Defect,Defect,Defect,Defect
30,Defect,Defect,No Defect,Defect,Defect
65,Defect,Defect,Defect,Defect,Defect
70,Defect,Defect,Defect,Defect,Defect
74,Defect,No Defect,No Defect,Defect,Defect


In [57]:
model_recall = (subset.model3 == subset.actual).mean()
baseline_recall = (subset.baseline_prediction == subset.actual).mean()

print(f'   model recall: {model_recall:.2%}')# .2% is simply rounding the decimal on the percentage 2 decimal places.
print(f'baseline recall: {baseline_recall:.2%}')

   model recall: 81.25%
baseline recall: 100.00%


In [58]:
print(classification_report(y_true, y_pred3, labels = ['No Defect', 'Defect']))

              precision    recall  f1-score   support

   No Defect       0.97      0.53      0.69       184
      Defect       0.13      0.81      0.23        16

    accuracy                           0.56       200
   macro avg       0.55      0.67      0.46       200
weighted avg       0.90      0.56      0.65       200



# Conclusion:
-  metric against the model3 has the highest percentage: 81.25%.

# 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

At Gives You Paws, anyone can upload pictures of their cats or dogs. 
The photos are then put through a two step process. 
- First an automated algorithm tags pictures as either a cat or a dog **(Phase I)**. 
- Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users **(Phase II)**.

Several models have already been developed with the data, and you can find their results here.: https://ds.codeup.com/data/gives_you_paws.csv

Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [60]:
paws = pd.read_csv('gives_you_paws.csv')
paws.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [62]:
paws.actual.value_counts(), paws.model1.value_counts(), paws.model2.value_counts(), paws.model3.value_counts(), paws.model4.value_counts()

(dog    3254
 cat    1746
 Name: actual, dtype: int64,
 dog    2937
 cat    2063
 Name: model1, dtype: int64,
 cat    3212
 dog    1788
 Name: model2, dtype: int64,
 dog    2508
 cat    2492
 Name: model3, dtype: int64,
 dog    4253
 cat     747
 Name: model4, dtype: int64)

In [63]:
paws['baseline_prediction'] = 'dog'

In [65]:
pd.crosstab(paws.actual, paws.model1)

model1,cat,dog
actual,,
cat,1423,323
dog,640,2614


In [66]:
pd.crosstab(paws.actual, paws.model2)

model2,cat,dog
actual,,
cat,1555,191
dog,1657,1597


In [67]:
pd.crosstab(paws.actual, paws.model3)

model3,cat,dog
actual,,
cat,893,853
dog,1599,1655


In [90]:
pd.crosstab(paws.actual, paws.model4)

model4,cat,dog
actual,,
cat,603,1143
dog,144,3110


In [91]:
p_true = paws.actual
p_pred1 = paws.model1
p_pred2 = paws.model2
p_pred3 = paws.model3
p_pred4 = paws.model4

### 4a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [94]:
#create new column with most common output as your baseline
paws['baseline_prediction'] = 'dog'

In [71]:
(paws.actual == paws.baseline_prediction).mean()

0.6508

In [72]:
model_accuracy = (paws.actual == paws.model1).mean()
baseline_accuracy = (paws.actual == paws.baseline_prediction).mean()

print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model accuracy: 80.74%
baseline accuracy: 65.08%


In [73]:
model_accuracy = (paws.actual == paws.model2).mean()
baseline_accuracy = (paws.actual == paws.baseline_prediction).mean()

print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model accuracy: 63.04%
baseline accuracy: 65.08%


In [74]:
model_accuracy = (paws.actual == paws.model3).mean()
baseline_accuracy = (paws.actual == paws.baseline_prediction).mean()

print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model accuracy: 50.96%
baseline accuracy: 65.08%


In [92]:
model_accuracy = (paws.actual == paws.model4).mean()
baseline_accuracy = (paws.actual == paws.baseline_prediction).mean()

print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model accuracy: 74.26%
baseline accuracy: 34.92%


In [ ]:
# I believe Model 1 had a higher percentage b'c the increased amt of TP & TN 
# as that is what they are counting algebraically in the numerator.

### 4b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend for Phase I? For Phase II?

In [ ]:
# I would recommend:
# PHASE I: Recall on Model4 with a baseline of 'dog' as they have the most amt of correctly predicted dogs.
#          Here we can gather the highest amt of dog pics
# PHASE II: Recall on Model2 with a baseline of 'cat' as they have the most amt of correctly predicted cats.
#           Here we can id & filter out the cat pics.

In [103]:
subset_d = paws[paws.actual == 'dog']
subset_d.head(2)

,actual,model1,model2,model3,model4,baseline_prediction
1,dog,dog,cat,cat,dog,cat
2,dog,cat,cat,cat,dog,cat


In [99]:
model_recall = (subset_d.actual == subset_d.model4).mean()
baseline_recall = (subset_d.actual == subset_d.baseline_prediction).mean()

print(f'   model recall: {model_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')

   model recall: 95.57%
baseline recall: 100.00%


In [100]:
paws['baseline_prediction'] = 'cat'

In [104]:
subset_c = paws[paws.actual == 'cat']
subset_c.head(2)

,actual,model1,model2,model3,model4,baseline_prediction
0,cat,cat,dog,cat,dog,cat
4,cat,cat,cat,dog,dog,cat


In [102]:
model_recall = (subset_c.actual == subset_c.model2).mean()
baseline_recall = (subset_c.actual == subset_c.baseline_prediction).mean()

print(f'   model recall: {model_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')

   model recall: 89.06%
baseline recall: 100.00%


### 4c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recomend for Phase I? For Phase II?

In [ ]:
# I would recommend:
# PHASE I: Recall on Model2 with a baseline of 'cat' as they have the most amt of correctly predicted cats.
#          Here we can gather the highest amt of cat pics
# PHASE II: Recall on Model4 with a baseline of 'dog' as they have the most amt of correctly predicted dogs.
#           Here we can id & filter out the dog pics.

In [85]:
paws['baseline_prediction'] = 'cat'

In [107]:
model_recall = (subset_c.actual == subset_c.model2).mean()
baseline_recall = (subset_c.actual == subset_c.baseline_prediction).mean()

print(f'   model recall: {model_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')

   model recall: 89.06%
baseline recall: 100.00%


In [108]:
paws['baseline_prediction'] = 'dog'

In [109]:
model_recall = (subset_d.actual == subset_d.model4).mean()
baseline_recall = (subset_d.actual == subset_d.baseline_prediction).mean()

print(f'   model recall: {model_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')

   model recall: 95.57%
baseline recall: 0.00%


# 5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

In [75]:
accuracy_score(p_true, p_pred1), accuracy_score(p_true, p_pred2), accuracy_score(p_true, p_pred3)

(0.8074, 0.6304, 0.5096)

- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

In [76]:
precision_score(p_true, p_pred1, labels = ['cat', 'dog'], average = None)

array([0.68977218, 0.89002383])

In [77]:

precision_score(p_true, p_pred2, labels = ['cat', 'dog'], average = None)

array([0.48412204, 0.89317673])

In [78]:
precision_score(p_true, p_pred3, labels = ['cat', 'dog'], average = None)

array([0.35834671, 0.65988836])

- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

In [79]:
recall_score(p_true, p_pred1, labels = ['cat', 'dog'], average = None)

array([0.81500573, 0.80331899])

In [80]:
recall_score(p_true, p_pred2, labels = ['cat', 'dog'], average = None)

array([0.8906071 , 0.49078058])

In [81]:
recall_score(p_true, p_pred3, labels = ['cat', 'dog'], average = None)

array([0.51145475, 0.50860479])

- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [82]:
print(classification_report(p_true, p_pred1, labels = ['cat', 'dog']))

              precision    recall  f1-score   support

         cat       0.69      0.82      0.75      1746
         dog       0.89      0.80      0.84      3254

    accuracy                           0.81      5000
   macro avg       0.79      0.81      0.80      5000
weighted avg       0.82      0.81      0.81      5000



In [83]:
print(classification_report(p_true, p_pred2, labels = ['cat', 'dog']))

              precision    recall  f1-score   support

         cat       0.48      0.89      0.63      1746
         dog       0.89      0.49      0.63      3254

    accuracy                           0.63      5000
   macro avg       0.69      0.69      0.63      5000
weighted avg       0.75      0.63      0.63      5000



In [84]:
print(classification_report(p_true, p_pred3, labels = ['cat', 'dog']))

              precision    recall  f1-score   support

         cat       0.36      0.51      0.42      1746
         dog       0.66      0.51      0.57      3254

    accuracy                           0.51      5000
   macro avg       0.51      0.51      0.50      5000
weighted avg       0.55      0.51      0.52      5000



In [105]:
print(classification_report(p_true, p_pred4, labels = ['cat', 'dog']))

              precision    recall  f1-score   support

         cat       0.81      0.35      0.48      1746
         dog       0.73      0.96      0.83      3254

    accuracy                           0.74      5000
   macro avg       0.77      0.65      0.66      5000
weighted avg       0.76      0.74      0.71      5000

